In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from models.reader import data_reader
import config
from utils.data_utils import dump_pkl, write_vocab, load_pkl, build_vocab, load_vocab
import numpy as np

In [4]:
col_sep=config.col_sep
data_path=config.train_seg_path
min_count=config.min_count

In [5]:
data_content, data_lbl = data_reader(data_path, col_sep)
word_lst = []
for i in data_content:
    word_lst.extend(i.split())

# word vocab
word_vocab = build_vocab(word_lst, min_count=min_count, sort=True, lower=True)

In [6]:
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2),vocabulary=word_vocab, sublinear_tf=True)
data_feature = vectorizer.fit_transform(data_content)

In [7]:
idf_featu = vectorizer.get_feature_names()

In [8]:
len(data_content)

4723

In [9]:
len(idf_featu)

184529

In [11]:
data_feature_arr = data_feature.toarray()

In [12]:
data_lbl = np.array(data_lbl)
data_content = np.array(data_content)

In [13]:
key_top = []

In [14]:
for ele in data_feature_arr[data_lbl=="1"]:
    ele_sort = sorted(zip(idf_featu, ele), key = lambda x:x[1], reverse = True)
    print(ele_sort[:20])
    key_top.append(ele_sort[:20])
    

[('色斑', 0.18169891678760772), ('电子游戏', 0.1477576305769424), ('jdb', 0.11471358009379988), ('q9', 0.10991497675901389), ('ag', 0.10328548248506113), ('ysb', 0.10268794800283967), ('淡化', 0.10199194268706209), ('拉霸', 0.1004469714063735), ('bsg', 0.09665785781543414), ('捕鱼', 0.09458258233783381), ('mg', 0.09062474327295615), ('pg', 0.08592593031609826), ('haba', 0.08143209838453601), ('新濠', 0.08099298105676296), ('牛b', 0.07995241898750935), ('png', 0.07949683759343955), ('pt', 0.07780313112200699), ('晒斑', 0.0777796919302485), ('抢庄', 0.07729093799166031), ('篮球明星', 0.07509905393134143)]
[('电子游戏', 0.15269660321830197), ('燕窝', 0.13437412078997069), ('jdb', 0.11854801647088198), ('q9', 0.11358901417399363), ('ag', 0.10673792125422303), ('ysb', 0.10612041347892401), ('拉霸', 0.10380452960317448), ('bsg', 0.09988876043250243), ('捕鱼', 0.09774411643046697), ('mg', 0.09365398193838188), ('pg', 0.08879810563020892), ('haba', 0.08415406207926573), ('新濠', 0.08370026673818298), ('牛b', 0.08262492265752588)

In [17]:
import sys
sys.getsizeof(vectorizer)/(1024**2)

5.340576171875e-05

## simhash

In [18]:
import jieba
import jieba.analyse
import numpy as np
import json
 
class simhash:
    def __init__(self,content):
        self.simhash=self.simhash(content)
 
    def __str__(self):
        return str(self.simhash)
 
    def simhash(self,content):
        #seg = jieba.cut(content)
        #jieba.analyse.set_stop_words('data/stop_words.txt_utf8')
        #keyWord = jieba.analyse.extract_tags(
        #    '|'.join(seg), topK=20, withWeight=True, allowPOS=())#在这里对jieba的tfidf.py进行了修改
        #将tags = sorted(freq.items(), key=itemgetter(1), reverse=True)修改成tags = sorted(freq.items(), key=itemgetter(1,0), reverse=True)
        #即先按照权重排序，再按照词排序
        keyWord = content
        keyList = []
        # print(keyWord)
        for feature, weight in keyWord:
            #print(feature + str(weight))
            weight = int(weight * 20)
            feature = self.string_hash(feature)
            temp = []
            for i in feature:
                if(i == '1'):
                    temp.append(weight)
                else:
                    temp.append(-weight)
            # print(temp)
            keyList.append(temp)
        list1 = np.sum(np.array(keyList), axis=0)
        #print(list1)
        if(keyList==[]): #编码读不出来
            return '00'   
        simhash = ''
        for i in list1:
            if(i > 0):
                simhash = simhash + '1'
            else:
                simhash = simhash + '0'
        return simhash
 
 
    def string_hash(self,source):
        if source == "":
            return 0
        else:
            x = hash(source)
            x = bin(x).replace('0b', '').zfill(128)[-128:]
            #print(source,x)
 
            return str(x)
#     def string_hash(self,source):
#         if source == "":
#             return 0
#         else:
#             x = ord(source[0]) << 7
#             m = 1000003
#             mask = 2 ** 128 - 1
#             for c in source:
#                 x = ((x * m) ^ ord(c)) & mask
#             x ^= len(source)
#             if x == -1:
#                 x = -2
#             x = bin(x).replace('0b', '').zfill(64)[-64:]
#             #print(source,x)
 
#             return str(x)
 
        '''
        以下是使用系统自带hash生成，虽然每次相同的会生成的一样，
        不过，对于不同的汉子产生的二进制，在计算海明码的距离会不一样，
        即每次产生的海明距离不一致
        所以不建议使用。
        '''
        # x=str(bin(hash(source)).replace('0b','').replace('-','').zfill(64)[-64:])
        # print(source,x,len(x))
        # return x
 
 
    def hammingDis(self,com):
        t1 = '0b' + self.simhash
        t2 = '0b' + com.simhash
        n=int(t1, 2) ^ int(t2, 2)
        i=0
        while n:
            n &= (n-1)
            i+=1
        return i
 


In [19]:
hash("什么")

4437859857787184855

In [47]:
key_top_1104 = []
for ele in data_feature_arr[(data_lbl=="2")|(data_lbl=="1")]:
    ele_sort = sorted(zip(idf_featu, ele), key = lambda x:x[1], reverse = True)
    #print(ele_sort[:20])
    key_top_1104.append(ele_sort[:200])

In [69]:
key_top_1104[6]

array([['藏红花', '0.18090719219062745'],
       ['电子游戏', '0.13637114324072544'],
       ['jdb', '0.10587353087312706'],
       ['q9', '0.10144471714507557'],
       ['ag', '0.09532610445673795'],
       ['ysb', '0.09477461713152761'],
       ['拉霸', '0.09270633450380462'],
       ['bsg', '0.08920921729741918'],
       ['红花', '0.08840011598556813'],
       ['捕鱼', '0.08729386654149035'],
       ['mg', '0.0836410261708598'],
       ['pg', '0.07930431278219072'],
       ['不能喝', '0.07828437495145585'],
       ['haba', '0.07515678418657143'],
       ['新濠', '0.07475150608505206'],
       ['牛b', '0.07379113173116611'],
       ['png', '0.07337065831597994'],
       ['pt', '0.07180747212436599'],
       ['抢庄', '0.0713347495822361'],
       ['篮球明星', '0.06931177632535812'],
       ['宙斯', '0.06893060624849902'],
       ['伊朗', '0.0688732235623022'],
       ['跳起来', '0.06854019115846686'],
       ['dt', '0.06766491351804006'],
       ['夺宝', '0.06761228901861337'],
       ['森林舞会', '0.06727684122548892'],


In [49]:
com = simhash(key_top_1104[1])

In [50]:
count = 0
acc_count = 0
diff_list = []
for text in key_top_1104:
    count += 1
    test_hash = simhash(text)
    diff = test_hash.hammingDis(com)
    #print(text)
    #print(diff)
    diff_list.append(diff)
    acc_count += 1
print(acc_count/float(count))

1.0


In [51]:
len(data_lbl[(data_lbl=="2") | (data_lbl=="1")])

4723

In [52]:
count = 0
for ele in zip(diff_list, data_lbl[(data_lbl=="1") | (data_lbl=="2")]):
    if ele[1] == "1":
        print(ele)
        count += 1
print(count)

(5, '1')
(0, '1')
(12, '1')
(11, '1')
(11, '1')
(10, '1')
(18, '1')
(17, '1')
(16, '1')
9


In [53]:
count = 0
for ele in zip(diff_list, data_lbl[(data_lbl=="2") | (data_lbl=="1")]):
    if ele[0] <= 19:
        print(ele)
        count += 1
print(count)

(5, '1')
(0, '1')
(12, '1')
(11, '1')
(11, '1')
(10, '1')
(18, '1')
(17, '1')
(16, '1')
(19, '2')
(19, '2')
(16, '2')
(19, '2')
(17, '2')
(19, '2')
15


In [59]:
9/15

0.6

In [55]:
diff_list = np.array(diff_list)

In [61]:
len(diff_list)

4723

In [56]:
key_top_1104  = np.array(key_top_1104)

In [57]:
count = 0
for ele in key_top_1104[diff_list<=19]:
    count += 1 
    if count >=10:
        print(ele)

[['魔兽世界' '0.1926413431322465']
 ['网游' '0.15797042415915677']
 ['天龙八部' '0.14604144033177682']
 ['绿色征途' '0.14030089042643887']
 ['激活码' '0.13988459023869568']
 ['礼包' '0.13713268819923322']
 ['仙剑奇侠传' '0.12970550674365036']
 ['征途' '0.12481421365493013']
 ['ol' '0.12132956878527895']
 ['乱斗' '0.12056594657488018']
 ['回合' '0.11747502963020788']
 ['online' '0.11530789187092189']
 ['藏宝阁' '0.11299298211328149']
 ['龙之谷' '0.11054455580350057']
 ['洪荒' '0.11054455580350057']
 ['大天使' '0.10837185088797824']
 ['魔力宝贝' '0.10533554209612486']
 ['炫舞' '0.10245505970175271']
 ['公测' '0.10240809622115069']
 ['寻仙' '0.09738657135055892']
 ['凡人' '0.09625661691926012']
 ['刀剑' '0.09560906870088925']
 ['梦幻西游' '0.09302968112393345']
 ['充值' '0.09234566799514686']
 ['修真' '0.09230066204283298']
 ['好玩' '0.09147015048207999']
 ['加点' '0.08963205161478432']
 ['仙之痕' '0.08956988407801098']
 ['火影忍者' '0.08885130452733234']
 ['天使之剑' '0.08748982086088064']
 ['魔域' '0.08674359003743888']
 ['面对面' '0.08634529318595639']
 ['西游' '0.0861

In [64]:
data_content[diff_list<=19][data_lbl[diff_list<=19]=="2"]

array(['魔兽世界 战友 复活 傲剑 激活码 游戏 官方下载 网 枪神纪 下载 安装 绿色征途 免费 吗 龙武 装备 强化 攻略 大话西游3 下载 大型 网络 新 游戏 红色警戒 3 下载 诛仙   烈山 苍穹变 yy 礼包 网游 2014 排行榜 什么 客户端 游戏 好玩 百田 网 游戏 英雄 下载 征途 什么时候 开 新区 魔域 新 服 梦幻 大唐 怎么 加点 天龙八部 装备 瑞兰 2 号 多少 钱 一针 极光世界   官方网站 魔兽世界 转区 皇帝 传 游戏 古龙群侠传 手机 rpg 网游 风云 无双 刷 元宝 辅助 下载 大型 网游 洪荒 神话 游戏 龙之谷 官方 惊天动地 2 什么时候 公测 笑傲九天 游戏 下载 征途 大 更新 职业 免费   网游 可爱 情侣 梦幻岛 征途 2 迅雷 下载 魔兽世界 下载地址 梦幻西游 2   藏宝阁 q 炫舞 大闹天宫 ol 官网 弹弹堂 情侣 名字 大全 仙剑奇侠 游戏 仙之痕 魔力宝贝 手游 官网 魔兽世界 官方 论坛 魔域 防沉迷 yy 大天使 之 剑 激活码 炫舞 团 名字 大全 仙剑奇侠传 充值 成吉思汗 官网 天龙八部 3d 龙之谷 破晓奇兵 现在 什么 游戏 好 赚 人民币 生死狙击 礼包 领取 时空猎人 怎么 刷 金币 仙 魔 九界 ios 回合 制 游戏 魔兽世界 招募 流程 百战 天下   激活码 萌 系 q版 网游 网游   坐骑 东方 绿洲 有 什么 好玩 的 中国 刀剑 地 城 之 光 激活码 仙剑奇侠传 1 网游 航海 游戏 真三国无双 3 红色 密码 官方网站 梦三国 激活码 傲世 网游 下载 qq音乐 qq 炫舞 所有 音乐 2014 内测 网游 大天使 礼包 激活码 绿色征途 新区 大话 新手 序列号 斗战神 多玩 礼包 领取 不 花 时间 的 网游 itunes   store   账号 网游 什么 意思 网 游戏 排名 天龙八部 帐号 激活 免费   回合 制 网游 天下 足球 官方网站 魔兽世界 6.0 剧情 动画 风暴 英雄 激活码 联众世界 官方下载 客户端 网游 排行 魔兽世界 之 王 太古 仙域 手游 网游 之 战 御 天下 下载 魔兽世界 下载 慢 街机 群侠传 时装 魔兽世界 6.0   资料片 2.5d 游戏 排行榜 前十名 新 面对面 视频 游戏 官方 下载

In [63]:
data_lbl[diff_list<=19]

array(['1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2',
       '2', '2'], dtype='<U1')

In [58]:
15/16

0.9375

In [70]:
data_content[diff_list<=19][data_lbl[diff_list<=19]=="1"]

array(['108 好汉 188 188 体育 365 3655 3656 3659 38 238 威尼斯 3 分 pk10 9599 95998888 ag 埃及 奥秘 AG 电子 AG 电子 SB 57 AG 电子 Y MGG AG 电子 猛龙 传奇 AG 电子 神奇 宝石 ag 电子游戏 ag 电子游戏 网址 ag 电子 娱乐 ag 海洋 剧场 ag 杰克 高手 ag 金 拉霸 ag 空中 战争 AG 龙虎 AG 猛龙 传奇 AG 牛牛 AG 森林舞会 多 人 版 ag 水上乐园 ag 土地神 ag 西洋棋 ag 象棋 ag 新年 运 财 羊 ag 娱乐 ag 运 财 羊 游戏 AG 真人 bb 传统 BB 电子 BB 电子 连环 夺宝 BB 电子 糖果 派对 bb 电子游戏 bb 电子游戏 网址 bb 电子 娱乐 bb 发 大 财 BB 富贵 厅 bb 三国 bb 水果 乐园 BB 五行 BC 体育 BC 体育 篮球 bet 36 BG 畅游 厅 BG 牛牛 BOB 体育 BSG 电子 BSG 电子游戏 BSG 电子游戏 网址 BSG 电子 娱乐 Buffalo   Blitz Coral C Q9 蹦迪 C Q9 电子 C Q9 电子 777 C Q9 电子 非常 钻 C Q9 电子 鸿福齐天 C Q9 电子 皇 金 渔场 C Q9 电子游戏 C Q9 电子游戏 网址 C Q9 电子 娱乐 C Q9 非常 钻 C Q9 金鸡 报喜 C Q9 跳高 高 C Q9 跳起来 C Q9 跳起来 2 C Q9 武圣 C Q9 宙斯 Crown dafa DT 电子 DT 电子 king 5x 25 DT 电子 封神榜 DT 电子 疯狂 马戏团 DT 电子游戏 DT 电子游戏 网址 DT 电子 娱乐 DT 封神榜 GD 真人 GG 捕鱼 GG 捕鱼 天下 2 GPK 发 发 发 龙 HABA 电子 HABA 电子游戏 HABA 电子游戏 网址 HABA 电子 娱乐 IM 体育 JDB 财神 捕鱼 JDB 超级 牛B JDB 电子 JDB 电子 超级 牛B JDB 电子 好运 777 JDB 电子 聚宝盆 JDB 狗 来 富 JDB 聚宝盆 JDB 五龙 捕鱼 JDB 芝麻开门 K8 king 5x 25 KOK 体育 KY 看 牌 抢庄 牛牛 LOVE BET l

In [ ]:
for ele in zip(diff_list, data_lbl[(data_lbl=="2") | (data_lbl=="1104")]):
    if ele[1] == "1104":
        print(ele)

## TFIDF + jecard

In [ ]:
key_top_22_1104 = []
for ele in data_feature_arr[(data_lbl=="2201")|(data_lbl=="1104")]:
    ele_sort = sorted(zip(idf_featu, ele), key = lambda x:x[1], reverse = True)
    #print(ele_sort[:20])
    key_top_22_1104.append(ele_sort[:20])

In [ ]:
sentence1 = np.array(key_top_22_1104[6]).T[0]

In [ ]:
def compute_jaccard_similarity(sentence1, sentence2):
    word_set1 = set(sentence1)
    word_set2 = set(sentence2)

    return len(word_set1 & word_set2) / len(word_set1 | word_set2)

In [ ]:
sim_list = []
for sentence in key_top_22_1104:
    sim = compute_jaccard_similarity(sentence1, np.array(sentence).T[0])
    sim_list.append(sim)

In [ ]:
for ele in zip(sim_list, data_lbl[(data_lbl=="2201") | (data_lbl=="1104")]):
    if ele[1] == "2201":
        print(ele)

In [ ]:
count = 0
for ele in zip(sim_list, data_lbl[(data_lbl=="2201") | (data_lbl=="1104")]):
    if ele[0] > 0.08:
        print(ele)
        count += 1
print(count)

In [ ]:
17/18

##  TFIDF+JECARD 评估

## LDA

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
lda = LatentDirichletAllocation(n_components=3, learning_method='batch', max_iter=25, random_state=0)
lda_topics = lda.fit_transform(data_feature_arr[(data_lbl=="2201") | (data_lbl=="1104")])

In [ ]:
lda.components_

In [ ]:
lda_topics

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    #打印每个主题下权重较高的term
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

    #打印主题-词语分布矩阵
    print("#主题-词语分布矩阵: \n %s" %model.components_)

n_top_words=20
print_top_words(lda, idf_featu, n_top_words)

In [ ]:
lda_label = []
for ele in lda_topics:
    lda_label.append(np.array(ele).argsort()[::-1][0])
    

In [ ]:
for ele in zip(lda_label, data_lbl[(data_lbl=="2201") | (data_lbl=="1104")]):
    print(ele)

##  LDA准召 100%

In [ ]:
from sklearn.cluster import SpectralClustering
import numpy as np
import math

X = np.array([[185.4, 72.6],
    [155.0, 54.4],
    [170.2, 99.9],
    [172.2, 97.3],
    [157.5, 59.0],
    [190.5, 81.6],
    [188.0, 77.1],
    [167.6, 97.3],
    [172.7, 93.3],
    [154.9, 59.0]])

w, h = 10, 10;

#构建相似度矩阵，任意两个样本间的相似度= 100 - 两个样本的欧氏距离
Matrix = [[100- math.hypot(X[x][0]- X[y][0], X[x][1]- X[y][1]) for x in range(w)] for y in range(h)]

sc = SpectralClustering(3, affinity='precomputed', n_init=10)
sc.fit(Matrix)

print('spectral clustering')   
print(sc.labels_)
print(sc.affinity_matrix_)

In [ ]:
a = [1]
b = [5]

In [ ]:
a += b

In [ ]:
a